In [1]:
import requests 

docs_url = 'https://github.com/alexeygrigorev/llm-rag-workshop/raw/main/notebooks/documents.json'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [2]:
documents[2]


{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
 'section': 'General course-related questions',
 'question': 'Course - Can I still join the course after the start date?',
 'course': 'data-engineering-zoomcamp'}

In [3]:
from minsearch import AppendableIndex

index = AppendableIndex(
    text_fields=["question", "text", "section"],
    keyword_fields=["course"]
)

index.fit(documents)

In [5]:
def search(query):
    boost = {'question': 3.0, 'section': 0.5}

    results = index.search(
        query=query,
        filter_dict={'course': 'data-engineering-zoomcamp'},
        boost_dict=boost,
        num_results=5,
        output_ids=True
    )

    return results

In [8]:
question = 'Can I still join the course'

In [9]:
prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

<QUESTION>
{question}
</QUESTION>

<CONTEXT>
{context}
</CONTEXT>
""".strip()

def build_prompt(query, search_results):
    context = ""

    for doc in search_results:
        context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"
    
    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

In [10]:
search_results =  search(question)

In [11]:
prompt = build_prompt(question, search_results)

In [12]:
print(prompt)

You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

<QUESTION>
Can I still join the course
</QUESTION>

<CONTEXT>
section: General course-related questions
question: Course - Can I still join the course after the start date?
answer: Yes, even if you don't register, you're still eligible to submit the homeworks.
Be aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.

section: General course-related questions
question: Certificate - Can I follow the course in a self-paced mode and get a certificate?
answer: No, you can only get a certificate if you finish the course with a “live” cohort. We don't award certificates for the self-paced mode. The reason is you need to peer-review capstone(s) after submitting a project. You can only peer-review projects at the time the course is running.

section: General 

In [13]:
from openai import OpenAI
client = OpenAI()

def llm(prompt):
    response = client.chat.completions.create(
       model='gpt-3.5-turbo',
        messages=[{"role": "user", "content": prompt}]
    )
    return response.choices[0].message.content

def rag(query):
    search_results = search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [15]:
answer = llm(prompt)

In [16]:
print(answer)

Based on the information provided, you can still join the course even after the start date. However, it is important to note that there will be deadlines for turning in the final projects, so it is advised not to leave everything for the last minute.


In [17]:
rag('How do i run Kafka in Docker?')

'To run Kafka in Docker, you need to make sure that your Kafka broker docker container is working. Use the command `docker ps` to confirm its status. Then go to the docker compose yaml file folder and run `docker-compose up -d` to start all the instances.'

### 🧠 Agentic RAG: Context & Decision Flow

In a **Retrieval-Augmented Generation (RAG)** system, we enhance an LLM’s answers by adding external **context** (e.g., from FAQs).

In **Agentic RAG**, the model:

- 🧠 Decides when it needs more context  
- 🔍 Chooses whether to **search the FAQ** or use **its own knowledge**  
- 🔁 May chain actions: `SEARCH → BUILD CONTEXT → ANSWER`

---

### 🔁 Agentic Decision Flow

User Question
↓
Is context available?
↓
[Empty] → Action = “SEARCH”
↓
Retrieve FAQ results → Build Context
↓
Prompt LLM with Context
↓
Action = “ANSWER” → Return Final Answer

---

### 📦 What is Context?

The **context** is a formatted block created from the retrieved FAQ entries:

section: Getting Started
question: How do I enroll?
answer: Use the registration link before the course starts.

section: Slack Access
question: How do I join Slack?
answer: Use the invite link shared in the onboarding email.

This context is passed into the LLM prompt to help it **answer accurately using only verified sources**.

---


In [18]:
prompt_template = """
You're a course teaching assistant.

You're given a QUESTION from a course student and that you need to answer with your own knowledge and provided CONTEXT.
At the beginning the context is EMPTY.

<QUESTION>
{question}
</QUESTION>

<CONTEXT> 
{context}
</CONTEXT>

If CONTEXT is EMPTY, you can use our FAQ database.
In this case, use the following output template:

{{
"action": "SEARCH",
"reasoning": "<add your reasoning here>"
}}

If you can answer the QUESTION using CONTEXT, use this template:

{{
"action": "ANSWER",
"answer": "<your answer>",
"source": "CONTEXT"
}}

If the context doesn't contain the answer, use your own knowledge to answer the question

{{
"action": "ANSWER",
"answer": "<your answer>",
"source": "OWN_KNOWLEDGE"
}}
""".strip()

In [27]:
question = "how do I run docker on gentoo?"
context = "EMPTY"

prompt = prompt_template.format(question=question, context=context)
print(prompt)

answer = llm(prompt)
print(answer)

You're a course teaching assistant.

You're given a QUESTION from a course student and that you need to answer with your own knowledge and provided CONTEXT.
At the beginning the context is EMPTY.

<QUESTION>
how do I run docker on gentoo?
</QUESTION>

<CONTEXT> 
EMPTY
</CONTEXT>

If CONTEXT is EMPTY, you can use our FAQ database.
In this case, use the following output template:

{
"action": "SEARCH",
"reasoning": "<add your reasoning here>"
}

If you can answer the QUESTION using CONTEXT, use this template:

{
"action": "ANSWER",
"answer": "<your answer>",
"source": "CONTEXT"
}

If the context doesn't contain the answer, use your own knowledge to answer the question

{
"action": "ANSWER",
"answer": "<your answer>",
"source": "OWN_KNOWLEDGE"
}
<QUESTION>
how do I run docker on gentoo?
</QUESTION>

<CONTEXT> 
EMPTY
</CONTEXT>

{
"action": "SEARCH",
"reasoning": "Since the context is empty, I will search our FAQ database for information on running Docker on Gentoo."
}


<QUESTION>
how do I run docker on gentoo?
</QUESTION>

<CONTEXT> 
EMPTY
</CONTEXT>

{
"action": "SEARCH",
"reasoning": "Since the CONTEXT is empty, I will search our FAQ database to find a relevant answer on running Docker on Gentoo."
}


In [22]:
question = "how do I join the course?"
context = "EMPTY"

prompt = prompt_template.format(question=question, context=context)
answer = llm(prompt)
print(answer)

{
"action": "SEARCH",
"reasoning": "Since the CONTEXT is empty, I will search our FAQ database to find the answer to the student's question on how to join the course."
}


In [36]:
def build_context(search_results):
    context = ""

    for doc in search_results:
        context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"

    return context.strip()

In [37]:

answer = llm(prompt)
print(answer)

<QUESTION>
how do I run docker on gentoo?
</QUESTION>

<CONTEXT> 
EMPTY
</CONTEXT>

{
"action": "SEARCH",
"reasoning": "Since the context is empty, I will search our FAQ database for information on running Docker on Gentoo."
}


In [40]:
import re
import json

def extract_json(text):
    """Extracts the first JSON block from a string."""
    match = re.search(r'\{.*\}', text, re.DOTALL)
    if match:
        return match.group(0)
    else:
        raise ValueError("No JSON found in LLM output.")

def agentic_rag_v1(question):
    context = "EMPTY"
    prompt = prompt_template.format(question=question, context=context)

    print("🧠 Asking LLM (no context)...")
    raw_output = llm(prompt)
    print("🔵 LLM response (1st):")
    print(raw_output)

    try:
        answer = json.loads(extract_json(raw_output))
    except Exception as e:
        print("❌ JSON decode failed:", e)
        return raw_output

    if answer["action"] == "SEARCH":
        print("🟡 Searching FAQ...")
        search_results = search(question)
        context = build_context(search_results)

        prompt = prompt_template.format(question=question, context=context)

        print("🧠 Asking LLM (with context)...")
        raw_output = llm(prompt)
        print("🔵 LLM response (2nd):")
        print(raw_output)

        try:
            answer = json.loads(extract_json(raw_output))
        except Exception as e:
            print("❌ JSON decode failed (2nd call):", e)
            return raw_output

    print("✅ Final Answer:")
    print(answer["answer"])
    return answer

In [41]:
import json
agentic_rag_v1('how do I join the course?')


🧠 Asking LLM (no context)...
🔵 LLM response (1st):
<QUESTION>
how do I join the course?
</QUESTION>

<CONTEXT> 
EMPTY
</CONTEXT>

{
"action": "SEARCH",
"reasoning": "Since the context is empty, I will use our FAQ database to find the answer."
}
🟡 Searching FAQ...
🧠 Asking LLM (with context)...
🔵 LLM response (2nd):
{
"action": "ANSWER",
"answer": "To join the course, you need to register before the course starts using the provided registration link. Additionally, you should subscribe to the course public Google Calendar, join the course Telegram channel for announcements, and register in DataTalks.Club's Slack and join the channel.",
"source": "CONTEXT"
}
✅ Final Answer:
To join the course, you need to register before the course starts using the provided registration link. Additionally, you should subscribe to the course public Google Calendar, join the course Telegram channel for announcements, and register in DataTalks.Club's Slack and join the channel.


{'action': 'ANSWER',
 'answer': "To join the course, you need to register before the course starts using the provided registration link. Additionally, you should subscribe to the course public Google Calendar, join the course Telegram channel for announcements, and register in DataTalks.Club's Slack and join the channel.",
 'source': 'CONTEXT'}